In [1]:
from pycalphad import Database, Model, variables as v

In [2]:
from pycalphad import equilibrium
import numpy as np

TDB = """
 ELEMENT A    GRAPHITE                   12.011     1054.0      5.7423 !
 ELEMENT B   BCC_A2                     55.847     4489.0     27.2797 !
 TYPE_DEFINITION % SEQ * !
 PHASE TEST % 1 1 !
 CONSTITUENT TEST : A,B: !
"""

my_phases_rose = ['TEST']
comps = ['A', 'B']#, 'LI', 'BE', 'B', 'C', 'N', 'O', 'F']
comps = sorted(comps)
conds = dict({v.T: 1000, v.P: 101325})
#for comp in comps[1:]:
#    conds[v.X(comp)] = 1.0/float(len(comps))
conds[v.MU('A')] = -3000
#conds[v.X('A')] = 0.4
dbf = Database(TDB)
eqx = equilibrium(Database(TDB), comps, my_phases_rose, conds, calc_opts={'pdens': 10}, verbose=True)

Components: A B
Phases: TEST [done]
('Redundant phase:', CompositionSet(TEST, [ 0.44444444  0.55555556], NP=1.0, GM=-5711.742028618652))
Removing CompositionSet(TEST, [ 0.44444444  0.55555556], NP=nan, GM=-5711.742028618652)
('cur_conds', OrderedDict([('MU_A', array(-3000.0)), ('N', array(1.0)), ('P', array(101325.0)), ('T', array(1000.0))]))
('fixed_statevars', [('N', array(1.0)), ('P', array(101325.0)), ('T', array(1000.0))])
(array([0], dtype=int32), array([-3000.]))
('prob.x0', array([  1.00000000e+00,   1.01325000e+05,   1.00000000e+03,
         5.55555556e-01,   4.44444444e-01,   2.00000000e+00]))
('x_in', array([  1.00000000e+00,   1.01325000e+05,   1.00000000e+03,
         5.55555556e-01,   4.44444444e-01,   2.00000000e+00]))
('total_obj', -8090.150723904637)
('x_in', array([  1.00000001e+00,   1.01325000e+05,   1.00000000e+03,
         5.55555556e-01,   4.44444444e-01,   2.00000000e+00]))
('total_obj', -8090.150723904637)
('x_in', array([  1.00000000e+00,   1.01325001e+05,   1

In [3]:
eqx.MU

<xarray.DataArray 'MU' (N: 1, P: 1, T: 1, MU_A: 1, component: 2)>
array([[[[[-4397.750367, -4397.750367]]]]])
Coordinates:
  * T          (T) float64 1e+03
  * MU_A       (MU_A) float64 -3e+03
  * N          (N) float64 1.0
  * P          (P) float64 1.013e+05
  * component  (component) <U1 'A' 'B'

In [4]:
eqx.X

<xarray.DataArray 'X' (N: 1, P: 1, T: 1, MU_A: 1, vertex: 2, component: 2)>
array([[[[[[ 0.410762,  0.589238],
           [      nan,       nan]]]]]])
Coordinates:
  * T          (T) float64 1e+03
  * MU_A       (MU_A) float64 -3e+03
  * N          (N) float64 1.0
  * P          (P) float64 1.013e+05
  * vertex     (vertex) int64 0 1
  * component  (component) <U1 'A' 'B'

In [5]:
eqx.Y

<xarray.DataArray 'Y' (N: 1, P: 1, T: 1, MU_A: 1, vertex: 2, internal_dof: 2)>
array([[[[[[ 0.410762,  0.589238],
           [      nan,       nan]]]]]])
Coordinates:
  * T        (T) float64 1e+03
  * MU_A     (MU_A) float64 -3e+03
  * N        (N) float64 1.0
  * P        (P) float64 1.013e+05
  * vertex   (vertex) int64 0 1
Dimensions without coordinates: internal_dof